<a href="https://colab.research.google.com/github/0Tick/stable-diffusion-tools/blob/main/distilgpt2train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements
To use this notebook you will need to have around 8.7GB of free google drive space. (This is with a dataset of ~75k lines. In the end its just around 1GB but a checkpoint is saved every 500 steps. The larger the dataset the more steps. You can also just delete the older checkpoints while the notebook is running)

# Preparing the dataset
This is the trickiest part. You have to somehow gather a dataset of tags or sentences to train on. My recent datasets were about ~75k lines long.




If you got your file ready, upload it to your Google Drive account.

# Training
Training is very simple. 
*   Install the requirements (Run the first Cell)
*   Connect your Google Drive (Run the second Cell)

* *  Enter the Path to the dataset in your google drive (like "AI/dataset.txt") in the field **DATA**. Choose the amount of phrases **COUNT** (they are seperated by a space) you want to have per line in the Validation File (You can leave this at the default). If you have a custom seperator for your phrases you can enter it in the **SEPERATOR** Field.
* * Run the Cell and see if the line count is around 1/10 of the original file size then everything should work

  **OR**
* * Click **CUSTOM** and enter the path to your file with custom validation file where every Line is one test
* Enter the path to the  **OUTPUT** foler (in your google drive again) where you want to have the outputs stored into
* Start the Training by Running the Cell

# After Training
Donwload the "config.json", "merges.txt","pytorch_model.bin","tokenizer.json"" and "voacab.json".

Create a folder inside extensions/stable-diffusion-webui-promptgen/models with the name you want the model to be displayed in the "Promptgen" tab.

Copy the files you downloaded into the folder
After the training finished you can delete the checkpoint folders because the just take up space.

In [1]:
#@markdown Installing Requirements
!nvidia-smi
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/requirements.txt
!pip install -r requirements.txt
!pip install git+https://github.com/huggingface/transformers

Mon Feb 20 17:50:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    19W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@markdown Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

import random

DATA = "AI/train.txt" #@param{type:"string"}
COUNT = 3 #@param{type:"slider", min:1, max:10}
SEPERATOR = " " #@param{type:"string"}
#@markdown ---
#@markdown **OR**
#@markdown ---
CUSTOM = False #@param{type : "boolean"}
CUSTOM_FILE_PATH = "AI/custom-validation.txt" #@param {type:"string"}
if not CUSTOM:
  file = open(f"./drive/MyDrive/{DATA}")
  rawcont = file.read()
  file.close()
  linesold = rawcont.split("\n")
  outf = ""
  linecount = 0
  for i in range(int(len(linesold)/10)):
    words = linesold[i].split(SEPERATOR)
    neww =[words.pop(random.randint(0, len(words)-1)) for x in range(3)]
    outf += " ".join(neww)+"\n"
    linecount+=1
  test = open("./validate.txt", "w")
  test.write(outf)
  test.close()
  print(f"Number of validation Samples: {linecount}")
else:
  import shutil
  shutil.copy(f"./drive/MyDrive/{CUSTOM_FILE_PATH}","./validate.txt")




Number of validation Samples: 11821


In [4]:

OUTPUT = "AI/output" #@param{type:"string"}
!python run_clm.py \
    --model_name_or_path distilgpt2 \
    --train_file ./drive/MyDrive/{DATA} \
    --validation_file ./validate.txt \
    --per_device_train_batch_size 6 \
    --per_device_eval_batch_size 6 \
    --do_train \
    --do_eval \
    --output_dir ./drive/MyDrive/{OUTPUT}\

2023-02-20 17:51:16.510983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 17:51:18.075414: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-20 17:51:18.075557: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-20 17:51:18.075587: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War